In [1]:
import math
import cv2
import numpy as np
from time import time
import matplotlib.pyplot as plt

In [2]:
!pip install mediapipe opencv-python
!pip install opencv-python

  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\mahima joshi\\.conda\\envs\\mp_env\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\mahima joshi\\.conda\\envs\\mp_env\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [3]:
import mediapipe as mp

# Initializing mediapipe pose class
mp_pose = mp.solutions.pose

# Setting up the Pose function
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.8, model_complexity=2)

# Initializing mediapipe drawing class, useful for annotation
mp_drawing = mp.solutions.drawing_utils


In [ ]:
import cv2
import time
import mediapipe as mp
import math

# Initialize MediaPipe pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.4, model_complexity=1)

# Open webcam
video = cv2.VideoCapture(0)
if not video.isOpened():
    print("Error: Could not open video source.")
    exit()

cv2.namedWindow("Pose Detection", cv2.WINDOW_NORMAL)
video.set(3, 1280)
video.set(4, 960)

time1 = 0

#  Angle Calculation 
def calculateAngle(landmark1, landmark2, landmark3):
    x1, y1 = landmark1.x, landmark1.y
    x2, y2 = landmark2.x, landmark2.y
    x3, y3 = landmark3.x, landmark3.y

    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) -
                         math.atan2(y1 - y2, x1 - x2))
    if angle < 0:
        angle += 360
    return angle
    right_knee_angle = calculateAngle(
            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
            landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
        )

    if (
        165 < left_elbow_angle < 195 and
        165 < right_elbow_angle < 195 and
        80 < left_shoulder_angle < 110 and
        80 < right_shoulder_angle < 110 and
        165 < left_knee_angle < 195 and
        165 < right_knee_angle < 195
    ):
        label = 'T Pose'
        color = (0, 255, 0)

# ---- Almost T Pose with feedback ----
def classifyPose(landmarks, output_image, display=False):
    label = "Unknown Pose"
    feedback = ""
    color = (0, 0, 255)

    # --- Angle Calculations ---
    left_elbow_angle = calculateAngle(
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
    )

    right_elbow_angle = calculateAngle(
        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
        landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
        landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
    )

    left_shoulder_angle = calculateAngle(
        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    )

    right_shoulder_angle = calculateAngle(
        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
        landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
    )

    left_knee_angle = calculateAngle(
        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]
    )

    right_knee_angle = calculateAngle(
        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
        landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
        landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
    )


# --- T Pose ---
if (
    165 < left_elbow_angle < 195 and
    165 < right_elbow_angle < 195 and
    80 < left_shoulder_angle < 110 and
    80 < right_shoulder_angle < 110 and
    165 < left_knee_angle < 195 and
    165 < right_knee_angle < 195
):
    label = 'T Pose'
    color = (0, 255, 0)

elif (
    140 < left_elbow_angle < 210 and
    140 < right_elbow_angle < 210 and
    60 < left_shoulder_angle < 130 and
    60 < right_shoulder_angle < 130
):
    label = 'Almost T Pose'
    feedback_parts = []

    if not (165 < left_elbow_angle < 195):
        feedback_parts.append("Straighten left arm")

    if not (165 < right_elbow_angle < 195):
        feedback_parts.append("Straighten right arm")

    if not (80 < left_shoulder_angle < 110):
        feedback_parts.append("Adjust left shoulder level")

    if not (80 < right_shoulder_angle < 110):
        feedback_parts.append("Adjust right shoulder level")

    if not (165 < left_knee_angle < 195):
        feedback_parts.append("Straighten left leg")

    if not (165 < right_knee_angle < 195):
        feedback_parts.append("Straighten right leg")

    feedback = ', '.join(feedback_parts)
    color = (0, 165, 255)

# --- Warrior II Pose ---
elif (
    (165 < left_elbow_angle < 195 or 165 < right_elbow_angle < 195) and
    (80 < left_shoulder_angle < 110 and 80 < right_shoulder_angle < 110) and
    ((165 < left_knee_angle < 195 and 90 < right_knee_angle < 120) or
        (165 < right_knee_angle < 195 and 90 < left_knee_angle < 120))
):
    label = 'Warrior II Pose'
    color = (0, 255, 0)

elif (
    (150 < left_elbow_angle < 210 or 150 < right_elbow_angle < 210) and
    (65 < left_shoulder_angle < 125 and 65 < right_shoulder_angle < 125)
):
    label = 'Almost Warrior II'
    feedback = "Adjust shoulder alignment and bend one knee forward more."
    color = (0, 165, 255)

# --- Tree Pose ---
elif (
    (165 < left_knee_angle < 195 or 165 < right_knee_angle < 195) and
    (315 < left_knee_angle < 335 or 25 < right_knee_angle < 45)
):
    label = 'Tree Pose'
    color = (0, 255, 0)

elif (
    (140 < left_knee_angle < 200 or 140 < right_knee_angle < 200) and
    (300 < left_knee_angle < 345 or 15 < right_knee_angle < 60)
):
    label = 'Almost Tree Pose'
    feedback = "Straighten standing leg, lift bent leg higher."
    color = (0, 165, 255)

# Thumbs Up Pose 
elif (
    60 < right_elbow_angle < 100 and
    landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y <
    landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y and
    70 < right_shoulder_angle < 120
):
    label = 'Thumbs Up Pose'
    color = (0, 255, 0)

elif (
    40 < right_elbow_angle < 120 and
    landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y <
    landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y
):
    label = 'Almost Thumbs Up'
    feedback = "Raise arm a bit more for clearer thumbs up."
    color = (0, 165, 255)

# --- Display Label and Feedback ---
cv2.putText(output_image, label, (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, color, 2)
if feedback:
    cv2.putText(output_image, feedback, (10, 60), cv2.FONT_HERSHEY_PLAIN, 2, color, 2)

return output_image, label




NameError: name 'left_elbow_angle' is not defined

: 

In [ ]:
# Pose Detection Function
def detectPose(image, pose_model, display=False):
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose_model.process(rgb_image)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        return image, results.pose_landmarks.landmark
    else:
        return image, None

In [5]:
# ---- Real-time Loop ----
while video.isOpened():
    ok, frame = video.read()
    if not ok:
        break

    frame = cv2.flip(frame, 1)
    frame_height, frame_width, _ = frame.shape
    aspect_ratio = frame_width / frame_height
    new_width = int(aspect_ratio * 640)
    frame = cv2.resize(frame, (new_width, 640))

    frame, landmarks = detectPose(frame, pose_video, display=False)

    if landmarks is not None:
        frame, _ = classifyPose(landmarks, frame, display=False)

    time2 = time.time()
    if (time2 - time1) > 0:
        fps = 1.0 / (time2 - time1)
        cv2.putText(frame, f'FPS: {int(fps)}', (10, 60),
                    cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)
    time1 = time2

    cv2.imshow('Pose Detection', frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC key
        break

video.release()
cv2.destroyAllWindows()